In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[15]:


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import operator
import os
from PIL import Image
from tqdm import tqdm_notebook as tqdm
import random

from shutil import copyfile

# keras : librairie de deep learning
import keras
from keras.models import Sequential, Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, InputLayer, ReLU, AveragePooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import VGG16

Using TensorFlow backend.


In [2]:
def split_train_test_Hack(new_paths, path, airbus_planes = ['A320', 'A321', 'A350', 'A330'], split_limit = .7, 
                          s = 8):
    for plane in airbus_planes:
        os.makedirs(new_paths[0] + '/' + plane, exist_ok = True)
        os.makedirs(new_paths[1] + '/' + plane, exist_ok = True)
        
        picts = os.listdir(path + '/' + plane)
        picts = [pic for pic in picts if pic[-4:]=='.jpg']
        random.seed(a = s)
        random.shuffle(picts)
        
        for pict in picts[:int(split_limit*len(picts))]:
            copyfile(path + plane + '/' + pict, new_paths[0] + '/' + plane + '/' + pict)
        for pict in picts[int(split_limit*len(picts)):]:
            copyfile(path + plane + '/' + pict, new_paths[1] + '/' + plane + '/' + pict)
                

In [3]:
path = './../InputsHackathon/'
new_path_train = './../Split_data/Hack/data_train'
new_path_val = './../Split_data/Hack/data_val'
new_paths = [new_path_train, new_path_val]

In [4]:
split_train_test_Hack(new_paths, path, airbus_planes = ['A320', 'A380', 'A350', 'A330'], split_limit = 0.7,
                      s = None)

In [5]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies


train_generator = train_datagen.flow_from_directory(new_path_train,
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

test_generator = train_datagen.flow_from_directory(new_path_val,
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)


# # Transfer learning

# create the base pre-trained model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# add a global spatial average pooling layer
x = base_model.output


x = Conv2D(256, kernel_size=(3, 3))(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = Conv2D(256, kernel_size=(3, 3))(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = AveragePooling2D(pool_size=(2, 2))(x)

# vector
x = Flatten()(x)

# let's add a fully-connected layer
x = Dense(1024)(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = Dense(512)(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = Dense(256)(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)


# last layer which predict the class
predictions = Dense(4, activation = 'softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()




Found 1148 images belonging to 4 classes.
Found 941 images belonging to 4 classes.
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
__________________________________________

In [ ]:
# Define learning rate reducer
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', 
                              patience=2, 
                              verbose=1, 
                              factor=0.5, 
                              min_lr=0.00001)

# How to include it in your training

step_size_train=train_generator.n//train_generator.batch_size

model.fit_generator(generator=train_generator,
                    steps_per_epoch=step_size_train,
                    epochs=10,
                    validation_data=test_generator,
                    callbacks=[reduce_lr])  # here


Epoch 1/10
20/35 [================>.............] - ETA: 2:54 - loss: 1.4614 - accuracy: 0.4203